# Projet Knapsack Problem 0-1

**Équipe:** Chaabane, Arman, Bartosz, Ahmed

## Structure du projet

1. Infrastructure commune (Classes et structures de données)
2. Algorithmes implémentés
3. Système de benchmarking complet
4. Analyse comparative approfondie

---
## 1. Configuration et Imports

In [ ]:
import time
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from pathlib import Path
from collections import defaultdict
from types import SimpleNamespace

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Palette de couleurs bien distinctes pour les algorithmes
ALGO_COLORS = {
    'Brute Force': '#e41a1c',
    'Dynamic Programming': '#377eb8',
    'DP Top-Down': '#4daf4a',
    'Branch and Bound': '#984ea3',
    'Greedy Ratio': '#ff7f00', 
    'Greedy Value': '#ffff33', 
    'Greedy Weight': '#a65628', 
    'Fractional Knapsack': '#f781bf',
    'Randomized': '#999999', 
    'Genetic Algorithm': '#17becf',  
    'Genetic Adaptive': '#1f77b4',      
    'Simulated Annealing': '#d62728',   
    'SA Adaptive': '#ff9896',          
    'FTPAS (ε=0.1)': '#9467bd',         
    'FTPAS (ε=0.05)': '#8c564b',        
    'FTPAS Adaptive': '#e377c2',        
}

---
## 2. Structures de Données Communes

In [ ]:
class Item:
    """Représente un item avec son poids et sa valeur"""
    def __init__(self, item_id, weight, value):
        self.id = item_id
        self.weight = weight
        self.value = value
    
    def ratio(self):
        return self.value / self.weight if self.weight > 0 else 0
    
    def __repr__(self):
        return f"Item({self.id}, w={self.weight}, v={self.value})"


class Problem:
    """Représente une instance du problème de knapsack"""
    def __init__(self, items, capacity):
        self.items = items
        self.capacity = capacity
        self.n = len(items)


class Solution:
    """Représente une solution au problème"""
    def __init__(self, selected_items, total_value, total_weight, time_taken):
        self.selected_items = selected_items
        self.total_value = total_value
        self.total_weight = total_weight
        self.time = time_taken
        self.usage_percent = (total_weight / 1.0) * 100  # Sera mis à jour

---
## 3. Parsing et Gestion des Benchmarks

In [ ]:
def parse_benchmark_file(filepath):
    """Parse un fichier benchmark .kp
    
    Format: 
    - Ligne 1: n capacity (séparés par espace)
    - Lignes suivantes: value weight (profit puis poids)
    """
    try:
        with open(filepath, 'r') as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]
        
        # Première ligne contient n et capacity séparés par espace
        first_line_parts = lines[0].split()
        n = int(first_line_parts[0])
        capacity = int(first_line_parts[1])
        
        items = []
        for i in range(n):
            parts = lines[1 + i].split()
            value = int(parts[0])  # profit/value
            weight = int(parts[1])  # poids
            items.append(Item(i, weight, value))
        
        return Problem(items, capacity)
    except Exception as e:
        print(f"Erreur parsing {filepath}: {e}")
        return None

---
## 4. Algorithmes Implémentés

### 4.1 Brute Force

In [ ]:
def brute_force(problem):
    """Algorithme exhaustif O(2^n)"""
    start_time = time.time()
    
    best_value = 0
    best_weight = 0
    best_items = []
    
    for size in range(problem.n + 1):
        for combo in combinations(range(problem.n), size):
            total_weight = sum(problem.items[i].weight for i in combo)
            total_value = sum(problem.items[i].value for i in combo)
            
            if total_weight <= problem.capacity and total_value > best_value:
                best_value = total_value
                best_weight = total_weight
                best_items = list(combo)
    
    time_taken = time.time() - start_time
    sol = Solution(best_items, best_value, best_weight, time_taken)
    sol.usage_percent = (best_weight / problem.capacity) * 100 if problem.capacity > 0 else 0
    return sol

### 4.2 Programmation Dynamique

In [ ]:
def dynamic_programming(problem):
    """Programmation dynamique O(n x C) avec protection"""
    start_time = time.time()
    
    n = problem.n
    C = problem.capacity
    
    total_cells = n * C
    if total_cells > 10_000_000:
        print(f"DP Skip: matrice trop grande ({n}×{C:,} = {total_cells:,})")
        return None
    
    estimated_mb = (total_cells * 8) / (1024 * 1024)
    if estimated_mb > 500:
        print(f"DP Skip: mémoire > 500 MB ({estimated_mb:.0f} MB)")
        return None
    
    dp = [[0 for _ in range(C + 1)] for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        item = problem.items[i - 1]
        for w in range(C + 1):
            dp[i][w] = dp[i - 1][w]
            if item.weight <= w:
                dp[i][w] = max(dp[i][w], dp[i - 1][w - item.weight] + item.value)
    
    # Reconstruction
    selected = []
    w = C
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected.append(i - 1)
            w -= problem.items[i - 1].weight
    
    total_value = dp[n][C]
    total_weight = sum(problem.items[i].weight for i in selected)
    
    time_taken = time.time() - start_time
    sol = Solution(selected, total_value, total_weight, time_taken)
    sol.usage_percent = (total_weight / problem.capacity) * 100 if problem.capacity > 0 else 0
    return sol

### 4.2.1 Programmation Dynamique Top-Down

In [ ]:
def dynamic_programming_topdown(problem):
    """
    Programmation dynamique Top-Down avec mémoïsation
    
    Complexité temporelle: O(n × C)
    Complexité spatiale: O(n × C) pour le cache + O(n) pour la pile de récursion
    
    Avantages par rapport à Bottom-Up:
    - Ne calcule que les sous-problèmes nécessaires
    - Plus intuitif (suit la définition récursive)
    - Peut être plus rapide si tous les sous-problèmes ne sont pas nécessaires
    
    Returns:
        Solution object
    """
    import sys
    start_time = time.time()
    
    n = problem.n
    C = problem.capacity
    items = problem.items
    
    # Protection contre les grandes instances
    total_cells = n * C
    if total_cells > 10_000_000:
        print(f"DP Top-Down Skip: cache trop grand ({n}×{C:,} = {total_cells:,})")
        return None
    
    # Augmenter la limite de récursion si nécessaire
    old_limit = sys.getrecursionlimit()
    if n + 100 > old_limit:
        sys.setrecursionlimit(max(n + 100, old_limit))
    
    # Cache pour mémoïsation: memo[i][w] = valeur max avec items 0..i-1 et capacité w
    memo = {}
    
    def knapsack(i, w):
        """
        Retourne la valeur maximale possible avec les items 0..i-1 et capacité w
        """
        # Cas de base
        if i == 0 or w == 0:
            return 0
        
        # Vérifier le cache
        if (i, w) in memo:
            return memo[(i, w)]
        
        item = items[i - 1]
        
        # Si l'item est trop lourd, on ne peut pas le prendre
        if item.weight > w:
            result = knapsack(i - 1, w)
        else:
            # Max entre prendre et ne pas prendre l'item
            not_take = knapsack(i - 1, w)
            take = knapsack(i - 1, w - item.weight) + item.value
            result = max(not_take, take)
        
        memo[(i, w)] = result
        return result
    
    # Calculer la valeur optimale
    try:
        best_value = knapsack(n, C)
    except RecursionError:
        print(f"DP Top-Down Skip: récursion trop profonde (n={n})")
        sys.setrecursionlimit(old_limit)
        return None
    
    # Reconstruction de la solution
    selected = []
    w = C
    for i in range(n, 0, -1):
        if w == 0:
            break
        item = items[i - 1]
        # Si la valeur change quand on exclut cet item, c'est qu'on l'a pris
        val_with = memo.get((i, w), 0)
        val_without = memo.get((i - 1, w), 0)
        if val_with != val_without:
            selected.append(i - 1)
            w -= item.weight
    
    total_weight = sum(items[i].weight for i in selected)
    
    # Restaurer la limite de récursion
    sys.setrecursionlimit(old_limit)
    
    time_taken = time.time() - start_time
    sol = Solution(selected, best_value, total_weight, time_taken)
    sol.usage_percent = (total_weight / problem.capacity) * 100 if problem.capacity > 0 else 0
    return sol

### 4.3 Branch and Bound

In [ ]:
def branch_and_bound(problem):
    """Branch and Bound avec élagage"""
    start_time = time.time()
    
    sorted_indices = sorted(range(problem.n), 
                          key=lambda i: problem.items[i].ratio(), 
                          reverse=True)
    
    best_value = 0
    best_solution = []
    
    def bound(level, current_weight, current_value):
        if current_weight >= problem.capacity:
            return 0
        
        value_bound = current_value
        total_weight = current_weight
        
        for i in range(level, problem.n):
            idx = sorted_indices[i]
            item = problem.items[idx]
            
            if total_weight + item.weight <= problem.capacity:
                total_weight += item.weight
                value_bound += item.value
            else:
                remaining = problem.capacity - total_weight
                value_bound += item.value * (remaining / item.weight)
                break
        
        return value_bound
    
    def branch(level, current_weight, current_value, current_items):
        nonlocal best_value, best_solution
        
        if level == problem.n:
            if current_value > best_value:
                best_value = current_value
                best_solution = current_items[:]
            return
        
        idx = sorted_indices[level]
        item = problem.items[idx]
        
        if current_weight + item.weight <= problem.capacity:
            new_value = current_value + item.value
            if bound(level + 1, current_weight + item.weight, new_value) > best_value:
                current_items.append(idx)
                branch(level + 1, current_weight + item.weight, new_value, current_items)
                current_items.pop()
        
        if bound(level + 1, current_weight, current_value) > best_value:
            branch(level + 1, current_weight, current_value, current_items)
    
    branch(0, 0, 0, [])
    
    total_weight = sum(problem.items[i].weight for i in best_solution)
    time_taken = time.time() - start_time
    
    sol = Solution(best_solution, best_value, total_weight, time_taken)
    sol.usage_percent = (total_weight / problem.capacity) * 100 if problem.capacity > 0 else 0
    return sol

### 4.4 Algorithmes Gloutons

In [ ]:
def greedy_by_value(problem):
    """Greedy par valeur décroissante"""
    start_time = time.time()
    
    sorted_items = sorted(enumerate(problem.items), key=lambda x: x[1].value, reverse=True)
    
    selected = []
    total_weight = 0
    total_value = 0
    
    for idx, item in sorted_items:
        if total_weight + item.weight <= problem.capacity:
            selected.append(idx)
            total_weight += item.weight
            total_value += item.value
    
    time_taken = time.time() - start_time
    sol = Solution(selected, total_value, total_weight, time_taken)
    sol.usage_percent = (total_weight / problem.capacity) * 100 if problem.capacity > 0 else 0
    return sol


def greedy_by_weight(problem):
    """Greedy par poids croissant"""
    start_time = time.time()
    
    sorted_items = sorted(enumerate(problem.items), key=lambda x: x[1].weight)
    
    selected = []
    total_weight = 0
    total_value = 0
    
    for idx, item in sorted_items:
        if total_weight + item.weight <= problem.capacity:
            selected.append(idx)
            total_weight += item.weight
            total_value += item.value
    
    time_taken = time.time() - start_time
    sol = Solution(selected, total_value, total_weight, time_taken)
    sol.usage_percent = (total_weight / problem.capacity) * 100 if problem.capacity > 0 else 0
    return sol


def greedy_by_ratio(problem):
    """Greedy par ratio valeur/poids décroissant"""
    start_time = time.time()
    
    sorted_items = sorted(enumerate(problem.items), key=lambda x: x[1].ratio(), reverse=True)
    
    selected = []
    total_weight = 0
    total_value = 0
    
    for idx, item in sorted_items:
        if total_weight + item.weight <= problem.capacity:
            selected.append(idx)
            total_weight += item.weight
            total_value += item.value
    
    time_taken = time.time() - start_time
    sol = Solution(selected, total_value, total_weight, time_taken)
    sol.usage_percent = (total_weight / problem.capacity) * 100 if problem.capacity > 0 else 0
    return sol

### 4.4.1 Fractional Knapsack (Sac à dos fractionnel)

In [ ]:
def fractional_knapsack(problem):
    """
    Fractional Knapsack - Algorithme glouton optimal pour le sac à dos fractionnel
    
    Complexité temporelle: O(n log n) pour le tri
    Complexité spatiale: O(n)
    
    Différence avec 0-1 Knapsack:
    - Permet de prendre une FRACTION d'un item
    - Solution optimale garantie (contrairement au 0-1)
    - Sert de borne supérieure pour le 0-1 Knapsack
    
    Stratégie: Trier par ratio valeur/poids décroissant et prendre 
    les items dans cet ordre (fractions si nécessaire)
    
    Returns:
        Solution object avec fraction_taken indiquant les fractions prises
    """
    start_time = time.time()
    
    n = problem.n
    capacity = problem.capacity
    items = problem.items
    
    # Trier les items par ratio valeur/poids décroissant
    sorted_items = sorted(enumerate(items), key=lambda x: x[1].ratio(), reverse=True)
    
    total_value = 0.0
    total_weight = 0.0
    selected = []  # Liste de tuples (index, fraction_prise)
    fractions = {}  # Pour stocker les fractions de chaque item
    
    remaining_capacity = capacity
    
    for idx, item in sorted_items:
        if remaining_capacity <= 0:
            break
            
        if item.weight <= remaining_capacity:
            # Prendre l'item entier
            selected.append(idx)
            fractions[idx] = 1.0
            total_value += item.value
            total_weight += item.weight
            remaining_capacity -= item.weight
        else:
            # Prendre une fraction de l'item
            fraction = remaining_capacity / item.weight
            fractions[idx] = fraction
            total_value += item.value * fraction
            total_weight += item.weight * fraction
            selected.append(idx)
            remaining_capacity = 0
    
    time_taken = time.time() - start_time
    
    # Créer la solution
    # Note: total_value peut être un float, on le garde ainsi pour la précision
    sol = Solution(selected, total_value, total_weight, time_taken)
    sol.usage_percent = (total_weight / capacity) * 100 if capacity > 0 else 0
    sol.fractions = fractions  # Stocker les fractions pour référence
    sol.is_fractional = True  # Marquer comme solution fractionnelle
    
    return sol


def fractional_knapsack_bound(problem):
    """
    Calcule uniquement la borne supérieure (valeur max du fractional knapsack)
    Utile pour Branch and Bound et comparaisons
    
    Returns:
        float: Valeur maximale possible (borne supérieure pour 0-1)
    """
    sorted_items = sorted(problem.items, key=lambda x: x.ratio(), reverse=True)
    
    total_value = 0.0
    remaining_capacity = problem.capacity
    
    for item in sorted_items:
        if remaining_capacity <= 0:
            break
        if item.weight <= remaining_capacity:
            total_value += item.value
            remaining_capacity -= item.weight
        else:
            total_value += item.value * (remaining_capacity / item.weight)
            break
    
    return total_value

### 4.5 Approche Randomisée

In [ ]:
def randomized_approach(problem, iterations=1000, seed=None):
    """Approche randomisée multi-start"""
    start_time = time.time()
    
    if seed is not None:
        random.seed(seed)
    
    best_value = 0
    best_weight = 0
    best_items = []
    
    for _ in range(iterations):
        indices = list(range(problem.n))
        random.shuffle(indices)
        
        selected = []
        total_weight = 0
        total_value = 0
        
        for idx in indices:
            item = problem.items[idx]
            if total_weight + item.weight <= problem.capacity:
                selected.append(idx)
                total_weight += item.weight
                total_value += item.value
        
        if total_value > best_value:
            best_value = total_value
            best_weight = total_weight
            best_items = selected
    
    time_taken = time.time() - start_time
    sol = Solution(best_items, best_value, best_weight, time_taken)
    sol.usage_percent = (best_weight / problem.capacity) * 100 if problem.capacity > 0 else 0
    return sol

### 4.6 Algorithme Génétique (Genetic Algorithm)

In [ ]:
def genetic_algorithm(problem, population_size=100, generations=50, 
                     crossover_rate=0.8, mutation_rate=0.02, 
                     elitism_count=5, seed=None):
    """

    Args:
        problem: Instance du problème (Problem object)
        population_size: Taille de la population (nombre de chromosomes)
        generations: Nombre de générations (itérations)
        crossover_rate: Probabilité de croisement (0.0 à 1.0)
        mutation_rate: Probabilité de mutation par gène (0.0 à 1.0)
        elitism_count: Nombre de meilleures solutions à conserver
        seed: Graine aléatoire pour reproductibilité
    
    Returns:
        Solution object
    """
    start_time = time.time()
    
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    
    n = problem.n
    capacity = problem.capacity
    items = problem.items
    
    # === 1. FONCTION DE FITNESS ===
    def fitness(chromosome):
        """Qualité d'un chromosome (solution)"""
        total_weight = sum(chromosome[i] * items[i].weight for i in range(n))
        total_value = sum(chromosome[i] * items[i].value for i in range(n))
        
        # Pénalité si capacité dépassée
        if total_weight > capacity:
            # Pénalité proportionnelle au dépassement
            penalty = (total_weight - capacity) * 10
            return max(0, total_value - penalty)
        return total_value
    
    # POPULATION INITIALE
    def create_initial_population():
        """Crée la population initiale avec différentes stratégies"""
        population = []
        
        # 50% solutions aléatoires
        for _ in range(population_size // 2):
            chromosome = [random.randint(0, 1) for _ in range(n)]
            population.append(chromosome)
        
        # 25% solutions greedy (ratio)
        sorted_indices = sorted(range(n), key=lambda i: items[i].ratio(), reverse=True)
        for _ in range(population_size // 4):
            chromosome = [0] * n
            weight = 0
            for idx in sorted_indices:
                if weight + items[idx].weight <= capacity and random.random() > 0.3:
                    chromosome[idx] = 1
                    weight += items[idx].weight
            population.append(chromosome)
        
        # 25% solutions avec densité variable
        for _ in range(population_size - len(population)):
            chromosome = [0] * n
            density = random.uniform(0.2, 0.8)
            weight = 0
            for i in range(n):
                if random.random() < density and weight + items[i].weight <= capacity:
                    chromosome[i] = 1
                    weight += items[i].weight
            population.append(chromosome)
        
        return population
    
    # SÉLECTION PAR TOURNOI
    def tournament_selection(population, fitnesses, tournament_size=3):
        """Sélectionne un individu par tournoi"""
        tournament_indices = random.sample(range(len(population)), tournament_size)
        tournament_fitnesses = [fitnesses[i] for i in tournament_indices]
        winner_idx = tournament_indices[tournament_fitnesses.index(max(tournament_fitnesses))]
        return population[winner_idx]
    
    # CROISEMENT (CROSSOVER)
    def crossover(parent1, parent2):
        """Croisement à deux points"""
        if random.random() > crossover_rate:
            return parent1[:], parent2[:]
        
        # Deux points de coupure
        point1 = random.randint(1, n - 2)
        point2 = random.randint(point1 + 1, n - 1)
        
        child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
        child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
        
        return child1, child2
    
    # MUTATION
    def mutate(chromosome):
        """Mutation par flip de bits"""
        mutated = chromosome[:]
        for i in range(n):
            if random.random() < mutation_rate:
                mutated[i] = 1 - mutated[i]  # Flip 0->1 ou 1->0
        return mutated
    
    # ALGORITHME PRINCIPAL
    population = create_initial_population()
    best_chromosome = None
    best_fitness = -1
    
    for gen in range(generations):
        # Évaluation de la population
        fitnesses = [fitness(chromo) for chromo in population]
        
        # Mise à jour de la meilleure solution
        gen_best_idx = fitnesses.index(max(fitnesses))
        gen_best_fitness = fitnesses[gen_best_idx]
        
        if gen_best_fitness > best_fitness:
            best_fitness = gen_best_fitness
            best_chromosome = population[gen_best_idx][:]
        
        # Tri par fitness (pour l'élitisme)
        sorted_indices = sorted(range(len(population)), key=lambda i: fitnesses[i], reverse=True)
        
        # Nouvelle génération
        new_population = []
        
        # Élitisme : garder les meilleurs
        for i in range(elitism_count):
            new_population.append(population[sorted_indices[i]][:])
        
        # Génération du reste de la population
        while len(new_population) < population_size:
            # Sélection
            parent1 = tournament_selection(population, fitnesses)
            parent2 = tournament_selection(population, fitnesses)
            
            # Croisement
            child1, child2 = crossover(parent1, parent2)
            
            # Mutation
            child1 = mutate(child1)
            child2 = mutate(child2)
            
            new_population.append(child1)
            if len(new_population) < population_size:
                new_population.append(child2)
        
        population = new_population
    
    # MEILLEURE SOLUTION
    selected_items = [i for i in range(n) if best_chromosome[i] == 1]
    total_value = sum(items[i].value for i in selected_items)
    total_weight = sum(items[i].weight for i in selected_items)
    
    time_taken = time.time() - start_time
    
    sol = Solution(selected_items, total_value, total_weight, time_taken)
    sol.usage_percent = (total_weight / capacity * 100) if capacity > 0 else 0
    
    return sol


def genetic_algorithm_adaptive(problem):
    """
    Version adaptative de l'algorithme génétique
    Ajuste les paramètres selon la taille du problème
    """
    n = problem.n
    
    if n <= 50:
        return genetic_algorithm(problem, population_size=50, generations=30, mutation_rate=0.03)
    elif n <= 100:
        return genetic_algorithm(problem, population_size=80, generations=40, mutation_rate=0.02)
    elif n <= 500:
        return genetic_algorithm(problem, population_size=100, generations=50, mutation_rate=0.02)
    elif n <= 1000:
        return genetic_algorithm(problem, population_size=120, generations=40, mutation_rate=0.01)
    else:
        return genetic_algorithm(problem, population_size=150, generations=30, mutation_rate=0.01)

### 4.6.1 Simulated Annealing (Recuit Simulé)

**Principe:** Inspiré du recuit métallurgique, l'algorithme explore l'espace des solutions en acceptant parfois des solutions moins bonnes pour échapper aux optima locaux. La probabilité d'accepter une mauvaise solution diminue avec la "température".

In [ ]:
def simulated_annealing(problem, initial_temp=1000, cooling_rate=0.995, 
                        min_temp=1, max_iterations=10000, seed=None):
    """
    Simulated Annealing (Recuit Simulé) pour le Knapsack 0-1
    
    Complexité temporelle: O(max_iterations × n)
    Complexité spatiale: O(n)
    
    Principe:
    - Commence avec une solution initiale (greedy)
    - À chaque itération, génère un voisin en flippant un bit
    - Accepte toujours les améliorations
    - Accepte les dégradations avec probabilité exp(-ΔE/T)
    - La température T diminue progressivement (refroidissement)
    
    Args:
        problem: Instance du problème
        initial_temp: Température initiale (contrôle l'exploration)
        cooling_rate: Taux de refroidissement (0.9 à 0.999)
        min_temp: Température minimale (critère d'arrêt)
        max_iterations: Nombre maximum d'itérations
        seed: Graine aléatoire pour reproductibilité
    
    Returns:
        Solution object
    """
    start_time = time.time()
    
    if seed is not None:
        random.seed(seed)
    
    n = problem.n
    capacity = problem.capacity
    items = problem.items
    
    # === FONCTION D'ÉVALUATION ===
    def evaluate(solution):
        """Calcule valeur et poids d'une solution (liste de 0/1)"""
        total_value = sum(solution[i] * items[i].value for i in range(n))
        total_weight = sum(solution[i] * items[i].weight for i in range(n))
        return total_value, total_weight
    
    def fitness(solution):
        """Fitness avec pénalité si capacité dépassée"""
        value, weight = evaluate(solution)
        if weight > capacity:
            # Pénalité proportionnelle au dépassement
            return value - (weight - capacity) * 10
        return value
    
    # === SOLUTION INITIALE (Greedy par ratio) ===
    current_solution = [0] * n
    sorted_indices = sorted(range(n), key=lambda i: items[i].ratio(), reverse=True)
    current_weight = 0
    for idx in sorted_indices:
        if current_weight + items[idx].weight <= capacity:
            current_solution[idx] = 1
            current_weight += items[idx].weight
    
    current_fitness = fitness(current_solution)
    best_solution = current_solution[:]
    best_fitness = current_fitness
    
    # === BOUCLE PRINCIPALE ===
    temperature = initial_temp
    iteration = 0
    
    while temperature > min_temp and iteration < max_iterations:
        # Générer un voisin en flippant un bit aléatoire
        neighbor = current_solution[:]
        flip_idx = random.randint(0, n - 1)
        neighbor[flip_idx] = 1 - neighbor[flip_idx]
        
        neighbor_fitness = fitness(neighbor)
        
        # Calculer la différence d'énergie
        delta = neighbor_fitness - current_fitness
        
        # Décision d'acceptation
        if delta > 0:
            # Amélioration : toujours accepter
            current_solution = neighbor
            current_fitness = neighbor_fitness
        else:
            # Dégradation : accepter avec probabilité exp(delta/T)
            acceptance_prob = math.exp(delta / temperature)
            if random.random() < acceptance_prob:
                current_solution = neighbor
                current_fitness = neighbor_fitness
        
        # Mettre à jour la meilleure solution
        if current_fitness > best_fitness:
            # Vérifier que la solution est valide
            _, weight = evaluate(current_solution)
            if weight <= capacity:
                best_solution = current_solution[:]
                best_fitness = current_fitness
        
        # Refroidissement
        temperature *= cooling_rate
        iteration += 1
    
    # === RÉSULTAT FINAL ===
    # S'assurer que la meilleure solution est valide
    selected_items = [i for i in range(n) if best_solution[i] == 1]
    total_value = sum(items[i].value for i in selected_items)
    total_weight = sum(items[i].weight for i in selected_items)
    
    # Réparer si nécessaire (retirer des items si surpoids)
    if total_weight > capacity:
        # Trier par ratio croissant et retirer
        selected_sorted = sorted(selected_items, key=lambda i: items[i].ratio())
        while total_weight > capacity and selected_sorted:
            remove_idx = selected_sorted.pop(0)
            total_weight -= items[remove_idx].weight
            total_value -= items[remove_idx].value
            selected_items.remove(remove_idx)
    
    time_taken = time.time() - start_time
    
    sol = Solution(selected_items, total_value, total_weight, time_taken)
    sol.usage_percent = (total_weight / capacity * 100) if capacity > 0 else 0
    sol.iterations = iteration
    sol.final_temperature = temperature
    
    return sol


def simulated_annealing_adaptive(problem):
    """
    Version adaptative de Simulated Annealing
    Ajuste les paramètres selon la taille du problème
    """
    n = problem.n
    
    if n <= 50:
        return simulated_annealing(problem, initial_temp=500, cooling_rate=0.99, max_iterations=5000)
    elif n <= 200:
        return simulated_annealing(problem, initial_temp=1000, cooling_rate=0.995, max_iterations=10000)
    elif n <= 1000:
        return simulated_annealing(problem, initial_temp=2000, cooling_rate=0.997, max_iterations=15000)
    else:
        return simulated_annealing(problem, initial_temp=5000, cooling_rate=0.999, max_iterations=20000)

### 4.7 FTPAS

In [ ]:
def ftpas(problem, epsilon=0.1):
    """
    Complexité: O(n³/ε)
    
    Args:
        problem: Instance du problème (Problem object)
        epsilon: Paramètre d'approximation (0 < ε < 1)
                Plus ε est petit, meilleure est l'approximation (mais plus lent)
    
    Returns:
        Solution object
    """
    start_time = time.time()
    
    n = problem.n
    items = problem.items
    capacity = problem.capacity
    
    if epsilon <= 0 or epsilon >= 1:
        print(f"FTPAS: epsilon doit être dans ]0,1[, reçu {epsilon}")
        return None
    
    v_max = max(item.value for item in items)
    
    # Facteur de scaling
    # K = (ε * v_max) / n
    K = (epsilon * v_max) / n
    
    # Si K trop petit, problèmes numériques
    if K < 1e-10:
        K = 1e-10
    
    # Créer les valeurs scalées (arrondi inférieur)
    scaled_items = []
    for item in items:
        scaled_value = math.floor(item.value / K)
        scaled_items.append({
            'original_idx': item.id,
            'weight': item.weight,
            'value': item.value,
            'scaled_value': scaled_value
        })
    
    V_scaled = sum(si['scaled_value'] for si in scaled_items)
    
    if V_scaled > 1_000_000:
        print(f"FTPAS Skip: V_scaled trop grand ({V_scaled:,})")
        return None
    
    # Protection supplémentaire
    estimated_mb = (n * V_scaled * 8) / (1024 * 1024)
    if estimated_mb > 200:  # Max 200 MB
        print(f"FTPAS Skip: mémoire estimée trop grande ({estimated_mb:.0f} MB)")
        return None
    
    # DP sur les valeurs scalées
    # dp[i][v] = poids minimum pour obtenir exactement la valeur scalée v avec les i premiers items
    INF = float('inf')
    dp = [[INF for _ in range(int(V_scaled) + 1)] for _ in range(n + 1)]
    dp[0][0] = 0
    
    for i in range(1, n + 1):
        si = scaled_items[i - 1]
        for v in range(int(V_scaled) + 1):
            # Ne pas prendre l'item i
            dp[i][v] = dp[i-1][v]
            
            # Prendre l'item i
            if v >= si['scaled_value']:
                prev_v = v - si['scaled_value']
                if dp[i-1][prev_v] != INF:
                    new_weight = dp[i-1][prev_v] + si['weight']
                    if new_weight <= capacity:
                        dp[i][v] = min(dp[i][v], new_weight)
    
    best_scaled_value = 0
    for v in range(int(V_scaled) + 1):
        if dp[n][v] <= capacity:
            best_scaled_value = v
    
    # Reconstruction de la solution
    selected = []
    v = best_scaled_value
    for i in range(n, 0, -1):
        if v == 0:
            break
        si = scaled_items[i - 1]
        prev_v = v - si['scaled_value']
        if prev_v >= 0 and dp[i-1][prev_v] != INF:
            if dp[i][v] == dp[i-1][prev_v] + si['weight']:
                selected.append(si['original_idx'])
                v = prev_v
    
    # Calculer la valeur non scalée de la solution
    total_value = sum(items[idx].value for idx in selected)
    total_weight = sum(items[idx].weight for idx in selected)
    
    time_taken = time.time() - start_time
    
    # Créer l'objet Solution
    sol = SimpleNamespace(
        selected_items=selected,
        total_value=total_value,
        total_weight=total_weight,
        time=time_taken,
        usage_percent=(total_weight / capacity * 100) if capacity > 0 else 0,
        epsilon=epsilon,
        scaling_factor=K
    )
    
    return sol


def ftpas_adaptive(problem, time_budget=None):
    """
    Ajuste epsilon selon la taille du problème
    
    Args:
        problem: Instance du problème
        time_budget: Budget de temps optionnel (non utilisé pour l'instant)
    
    Returns:
        Solution object
    """
    n = problem.n
    
    if n <= 50:
        epsilon = 0.1
    elif n <= 100:
        epsilon = 0.2
    elif n <= 500:
        epsilon = 0.3
    else:
        epsilon = 0.5  # Très rapide pour grandes instances
    
    return ftpas(problem, epsilon)

---
## 5. Système de Benchmarking

### 5.1 Configuration des Tests

In [ ]:
def discover_benchmarks(base_path='benchmarks'):
    """
    Découvre automatiquement tous les fichiers benchmark disponibles.
    
    Returns:
        dict: Structure contenant les benchmarks organisés par catégorie
    """
    base = Path(base_path)
    
    if not base.exists():
        print(f"Dossier '{base_path}' non trouvé")
        return None
    
    structure = {
        'base_path': str(base),
        'benchmarks': {}
    }
    
    # Scanner les dossiers low_dimension et large_scale
    categories = ['low_dimension', 'large_scale']
    
    for category in categories:
        category_path = base / category
        if not category_path.exists():
            continue
            
        for file_path in category_path.glob('*.txt'):
            filename = file_path.name
            
            # Extraire les informations du nom de fichier
            # Format low_dimension: f1_l-d_kp_10_269.txt -> n=10, capacity=269
            # Format large_scale: knapPI_1_100_1000_1.txt -> n=100, capacity=1000
            
            if category == 'low_dimension':
                # Format: fX_l-d_kp_N_C.txt
                parts = filename.replace('.txt', '').split('_')
                try:
                    n = int(parts[3])
                    cap = int(parts[4])
                except (IndexError, ValueError):
                    continue
            else:
                # Format: knapPI_X_N_C_1.txt
                parts = filename.replace('.txt', '').split('_')
                try:
                    n = int(parts[2])
                    cap = int(parts[3])
                except (IndexError, ValueError):
                    continue
            
            key = f"{category}_{filename}"
            structure['benchmarks'][key] = {
                'path': str(file_path),
                'correlation': category,
                'size': f"n={n}",
                'capacity': f"c={cap}",
                'n': n,
                'capacity_value': cap
            }
    
    print(f"Découvert {len(structure['benchmarks'])} benchmarks")
    return structure


# Initialiser la structure des benchmarks
BENCHMARK_STRUCTURE = discover_benchmarks()

if BENCHMARK_STRUCTURE:
    print(f"\nCatégories disponibles:")
    categories = {}
    for key, info in BENCHMARK_STRUCTURE['benchmarks'].items():
        cat = info['correlation']
        if cat not in categories:
            categories[cat] = 0
        categories[cat] += 1
    for cat, count in categories.items():
        print(f"  - {cat}: {count} fichiers")

In [ ]:
# Définition des algorithmes à tester
ALL_ALGORITHMS = [
    ('Brute Force', brute_force, 20),
    ('Dynamic Programming', dynamic_programming, 1000),
    ('DP Top-Down', dynamic_programming_topdown, 1000),
    ('Branch and Bound', branch_and_bound, 200),
    ('Greedy Ratio', greedy_by_ratio, float('inf')),
    ('Greedy Value', greedy_by_value, float('inf')),
    ('Greedy Weight', greedy_by_weight, float('inf')),
    ('Fractional Knapsack', fractional_knapsack, float('inf')),
    ('Randomized', lambda p: randomized_approach(p, iterations=100), float('inf')),
    ('Genetic Algorithm', lambda p: genetic_algorithm(p, population_size=100, generations=50), float('inf')),
    ('Genetic Adaptive', genetic_algorithm_adaptive, float('inf')),
    ('Simulated Annealing', lambda p: simulated_annealing(p, initial_temp=1000, cooling_rate=0.995), float('inf')),
    ('SA Adaptive', simulated_annealing_adaptive, float('inf')),
    ('FTPAS (ε=0.1)', lambda p: ftpas(p, epsilon=0.1), float('inf')),
    ('FTPAS (ε=0.05)', lambda p: ftpas(p, epsilon=0.05), float('inf')),
    ('FTPAS Adaptive', ftpas_adaptive, float('inf')),
]


def should_run_algorithm(algo_name, n, max_n):
    """Détermine si un algorithme doit être exécuté selon la taille"""
    return n <= max_n


def run_benchmark(benchmark_info, algorithms=None, timeout=300):
    """
    Exécute un benchmark sur un fichier.
    
    Returns:
        dict: Résultats pour chaque algorithme
    """
    if algorithms is None:
        algorithms = ALL_ALGORITHMS
    
    problem = parse_benchmark_file(benchmark_info['path'])
    if problem is None:
        return None
    
    results = {
        'info': benchmark_info,
        'n': problem.n,
        'capacity': problem.capacity,
        'algorithms': {}
    }
    
    for algo_name, algo_func, max_n in algorithms:
        if not should_run_algorithm(algo_name, problem.n, max_n):
            results['algorithms'][algo_name] = {'skipped': True, 'reason': f'n={problem.n} > max_n={max_n}'}
            continue
        
        try:
            #GESTION DES ERREURS
            start = time.time()
            sol = algo_func(problem)
            elapsed = time.time() - start
            
            # Vérifier timeout
            if elapsed > timeout:
                results['algorithms'][algo_name] = {
                    'skipped': True,
                    'reason': f'timeout (>{timeout}s)'
                }
                print(f"  ⏱️  {algo_name}: timeout ({elapsed:.1f}s)")
                continue
            
            # Vérifier si algo a retourné None (protection interne)
            if sol is None:
                results['algorithms'][algo_name] = {
                    'skipped': True,
                    'reason': 'protection_triggered'
                }
                continue
            # FIN GESTION ERREURS
            
            results['algorithms'][algo_name] = {
                'value': sol.total_value,
                'weight': sol.total_weight,
                'time': sol.time,
                'usage': sol.usage_percent,
                'items_count': len(sol.selected_items),
                'skipped': False
            }
            
        except KeyboardInterrupt:
            print(f"\nInterruption manuelle sur {algo_name}")
            results['algorithms'][algo_name] = {
                'skipped': True,
                'reason': 'interrupted'
            }
            continue  # Continuer avec les autres algorithmes
            
        except MemoryError:
            results['algorithms'][algo_name] = {
                'skipped': True,
                'reason': 'memory_error'
            }
            print(f"{algo_name}: Mémoire insuffisante")
            
        except Exception as e:
            results['algorithms'][algo_name] = {
                'skipped': True,
                'reason': str(e)
            }
            print(f"{algo_name}: {str(e)}")

        return results

### 5.2 Exécution Complète des Benchmarks

**Attention:** Cette cellule peut prendre plusieurs minutes selon le nombre de benchmarks.

In [ ]:
def run_all_benchmarks():
    """
    Exécute tous les benchmarks disponibles.
    Sauvegarde UNIQUEMENT à la fin, pas de sauvegarde partielle.
    Continue même en cas d'erreur sur un algorithme ou un benchmark.
    
    Returns:
        DataFrame avec tous les résultats
    """
    if BENCHMARK_STRUCTURE is None:
        print("Aucun benchmark disponible")
        return None
    
    all_results = []
    total = len(BENCHMARK_STRUCTURE['benchmarks'])
    
    print(f"Exécution de {total} benchmarks...")
    print("Le processus continue automatiquement même en cas d'erreur\n")
    
    for i, (key, bench_info) in enumerate(BENCHMARK_STRUCTURE['benchmarks'].items(), 1):
        print(f"\n[{i}/{total}] {bench_info['correlation']} | {bench_info['size']} | {bench_info['capacity']}")
        
        try:
            # Parser le problème
            problem = parse_benchmark_file(bench_info['path'])
            if problem is None:
                print(f"  ERREUR: Impossible de parser ce benchmark, skip")
                continue
            
            # Informations sur le problème
            print(f"  n={problem.n}, capacity={problem.capacity}")
            
            # Tester chaque algorithme
            for algo_name, algo_func, max_n in ALL_ALGORITHMS:
                # Vérifier si on doit exécuter cet algo
                if problem.n > max_n:
                    print(f"  SKIP {algo_name}: n={problem.n} > max_n={max_n}")
                    continue
                
                try:
                    # Exécuter l'algorithme
                    start_algo = time.time()
                    sol = algo_func(problem)
                    elapsed = time.time() - start_algo
                    
                    # Si l'algo prend plus de 5 minutes, on le note mais on garde le résultat
                    if elapsed > 300:
                        print(f"  WARNING {algo_name}: temps très long ({elapsed:.1f}s)")
                    
                    # Vérifier si l'algo a retourné None (protection interne)
                    if sol is None:
                        print(f"  SKIP {algo_name}: protection déclenchée")
                        continue
                    
                    # Enregistrer le résultat
                    row = {
                        'correlation': bench_info['correlation'],
                        'n': problem.n,
                        'capacity_type': bench_info['capacity'],
                        'capacity_value': problem.capacity,
                        'algorithm': algo_name,
                        'value': sol.total_value,
                        'time_ms': sol.time * 1000,
                        'usage_percent': sol.usage_percent,
                        'items_selected': len(sol.selected_items)
                    }
                    all_results.append(row)
                    
                    print(f"  OK {algo_name}: value={sol.total_value}, time={sol.time*1000:.2f}ms")
                
                except MemoryError:
                    print(f"  ERROR {algo_name}: Erreur mémoire")
                    continue
                
                except KeyboardInterrupt:
                    print(f"\n\nINTERRUPTION MANUELLE DÉTECTÉE")
                    print(f"Résultats collectés jusqu'ici: {len(all_results)} lignes")
                    
                    if len(all_results) > 0:
                        df = pd.DataFrame(all_results)
                        df.to_csv('benchmark_results_interrupted.csv', index=False)
                        print("💾 Sauvegarde d'urgence: 'benchmark_results_interrupted.csv'")
                        return df
                    else:
                        print("Aucun résultat à sauvegarder")
                        return None
                
                except Exception as e:
                    print(f"  ERROR {algo_name}: {str(e)}")
                    continue
        
        except Exception as e:
            print(f"  ERROR sur ce benchmark: {str(e)}")
            continue
    
    # FIN DE TOUS LES BENCHMARKS
    print(f"\n\n{'='*60}")
    print(f"TERMINÉ ! {len(all_results)} résultats collectés")
    print(f"{'='*60}\n")
    
    if len(all_results) == 0:
        print("ATTENTION: Aucun résultat collecté")
        return None
    
    # Créer le DataFrame final
    df = pd.DataFrame(all_results)
    
    # Sauvegarder
    try:
        df.to_csv('benchmark_results.csv', index=False)
        print("💾 Résultats sauvegardés: 'benchmark_results.csv'")
        
        # Afficher un résumé
        print(f"\nRésumé:")
        print(f"  - {df['algorithm'].nunique()} algorithmes testés")
        print(f"  - {df['n'].nunique()} tailles différentes")
        print(f"  - {df['correlation'].nunique()} types de corrélation")
        print(f"\nAperçu des résultats:")
        print(df.groupby('algorithm')['value'].agg(['count', 'mean', 'std']))
        
    except Exception as e:
        print(f"ERROR lors de la sauvegarde: {e}")
        print("Le DataFrame est quand même retourné")
    
    return df

### 5.3 Chargement des Résultats

Si les benchmarks ont déjà été exécutés, charger les résultats.

In [ ]:
# Charger les résultats
try:
    results_df = run_all_benchmarks()
    print(f"\nAperçu:")
except FileNotFoundError:
    print("Fichier 'benchmark_results.csv' non trouvé")
    print("Exécutez d'abord: results_df = run_all_benchmarks()")
    results_df = None

---
## 6. Visualisations simples

Ci-dessous :
- Graphe 1 : Valeur totale (axe Y) par algorithme (axe X), couleur = type de dataset (correlation).
- Graphe 2 : Temps d'exécution (ms) par algorithme, couleur = taille `n`.
- Graphe 3 : Scatter Temps (X) vs Valeur (Y), 1 point par algorithme, étiquette avec le nom.

In [ ]:
# Graphique 1 : Qualité (valeur totale) par algorithme, couleur = correlation
if results_df is not None:
    df = results_df.copy()
    # moyenne valeur par algorithme x correlation
    agg = df.groupby(['algorithm', 'correlation'])['value'].mean().reset_index()
    
    plt.figure(figsize=(12,6))
    # Palette pour large_scale et low_dimension
    palette_map = {'large_scale':'#1f77b4','low_dimension':'#ff7f0e'}
    # Fallback pour anciennes catégories si présentes
    palette_map.update({'uncorrelated':'#1f77b4','weakly_correlated':'#ff7f0e','strongly_correlated':'#2ca02c'})
    sns.barplot(data=agg, x='algorithm', y='value', hue='correlation', palette=palette_map)
    plt.xlabel('Algorithme')
    plt.ylabel('Valeur totale moyenne')
    plt.title('Valeur obtenue par algorithme (couleur = type de dataset)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("results_df non chargé. Exécutez d'abord la cellule qui charge 'benchmark_results.csv'.")

In [ ]:
# Graphique 2 : Temps d'exécution par taille n
if results_df is not None:
    df = results_df.copy()
    
    # Moyenne du temps par algorithme et taille n
    agg_time = df.groupby(['algorithm', 'n'])['time_ms'].mean().reset_index()
    
    # Trier par n
    agg_time = agg_time.sort_values('n')
    
    plt.figure(figsize=(12,6))
    
    # Utiliser lineplot avec algorithme comme hue (couleur)
    sns.lineplot(data=agg_time, x='n', y='time_ms', hue='algorithm', marker='o', palette='husl', linewidth=2, markersize=8)
    
    plt.xlabel('Taille du problème (n)')
    plt.ylabel('Temps moyen (ms)')
    plt.title('Temps d\'exécution par taille de problème (couleur = algorithme)')
    plt.yscale('log')
    plt.legend(title='Algorithme', bbox_to_anchor=(1.05,1), loc='upper left')
    plt.tight_layout()
    plt.show()
else:
    print("results_df non chargé. Exécutez d'abord la cellule qui charge 'benchmark_results.csv'.")

In [ ]:
# Graphique 3 : Scatter Temps (X) vs Valeur (Y), 1 point = 1 algorithme (moyennes), annoté
if results_df is not None:
    df = results_df.copy()
    # prendre la moyenne temps et valeur par algorithme
    summary = df.groupby('algorithm').agg({'time_ms':'mean','value':'mean'}).reset_index()
    
    plt.figure(figsize=(10,6))
    plt.scatter(summary['time_ms'], summary['value'], s=120, alpha=0.8)
    for i, row in summary.iterrows():
        plt.text(row['time_ms'], row['value'], row['algorithm'], fontsize=9,
                 verticalalignment='bottom', horizontalalignment='right')
    plt.xscale('log')
    plt.xlabel('Temps moyen (ms)')
    plt.ylabel('Valeur moyenne')
    plt.title('Compromis Temps vs Qualité (un point = un algorithme)')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("results_df non chargé. Exécutez d'abord la cellule qui charge 'benchmark_results.csv'.")

In [ ]:
if results_df is not None:
    df = results_df.copy()
    
    algorithms = df['algorithm'].unique()
    
    # couleurs pour les deux catégories principales
    colors_cat = {'large_scale':'#1f77b4','low_dimension':'#ff7f0e'}
    
    for algo in algorithms:
        algo_data = df[df['algorithm'] == algo].copy()
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        fig.suptitle(f'Performance : {algo}', fontsize=14, fontweight='bold')
        
        # Valeur par catégorie et taille n
        if 'category' in algo_data.columns:
            agg_value = algo_data.groupby(['category', 'n'])['value'].mean().reset_index()
            groups = agg_value['category'].unique()
        else:
            # fallback vers 'correlation' si présent
            agg_value = algo_data.groupby(['correlation', 'n'])['value'].mean().reset_index()
            agg_value = agg_value.rename(columns={'correlation':'category'})
            groups = agg_value['category'].unique()

        for cat in groups:
            cat_data = agg_value[agg_value['category'] == cat]
            axes[0].plot(cat_data['n'], cat_data['value'], marker='o', label=cat, linewidth=2,
                         color=colors_cat.get(cat, None))
        
        axes[0].set_xlabel('Taille n', fontsize=11)
        axes[0].set_ylabel('Valeur moyenne', fontsize=11)
        axes[0].set_title('Valeur par taille et catégorie')
        axes[0].legend()
        axes[0].grid(alpha=0.3)
        axes[0].set_xscale('log')
        
        # Temps d'exécution par taille n
        if 'category' in algo_data.columns:
            agg_time = algo_data.groupby(['category', 'n'])['time_ms'].mean().reset_index()
        else:
            agg_time = algo_data.groupby(['correlation', 'n'])['time_ms'].mean().reset_index()
            agg_time = agg_time.rename(columns={'correlation':'category'})

        for cat in agg_time['category'].unique():
            cat_data = agg_time[agg_time['category'] == cat]
            axes[1].plot(cat_data['n'], cat_data['time_ms'], marker='s', label=cat, linewidth=2,
                         color=colors_cat.get(cat, None))
        
        axes[1].set_xlabel('Taille n', fontsize=11)
        axes[1].set_ylabel('Temps moyen (ms)', fontsize=11)
        axes[1].set_title('Temps d\'exécution par taille')
        axes[1].legend()
        axes[1].grid(alpha=0.3)
        axes[1].set_xscale('log')
        axes[1].set_yscale('log')
        
        plt.tight_layout()
        plt.show()
        print(f"\n{'='*60}\n")
else:
    print("results_df non chargé. Exécutez d'abord la cellule qui charge 'benchmark_results.csv'.")

---
## 7. Analyse de Complexité

### 7.1 Complexité Théorique des Algorithmes

| Algorithme | Complexité Temporelle | Complexité Spatiale | Type |
|------------|----------------------|---------------------|------|
| **Brute Force** | $O(2^n)$ | $O(n)$ | Exact |
| **DP Bottom-Up** | $O(n \cdot C)$ | $O(n \cdot C)$ | Exact (pseudo-poly) |
| **DP Top-Down** | $O(n \cdot C)$ | $O(n \cdot C)$ | Exact (pseudo-poly) |
| **Branch & Bound** | $O(2^n)$ pire cas | $O(n)$ | Exact |
| **Greedy (ratio)** | $O(n \log n)$ | $O(n)$ | Approximation |
| **Fractional Knapsack** | $O(n \log n)$ | $O(n)$ | Exact (relaxé) |
| **Randomized** | $O(k \cdot n)$ | $O(n)$ | Heuristique |
| **Genetic Algorithm** | $O(g \cdot p \cdot n)$ | $O(p \cdot n)$ | Métaheuristique |
| **Simulated Annealing** | $O(i \cdot n)$ | $O(n)$ | Métaheuristique |
| **FPTAS** | $O(n^2 / \varepsilon)$ | $O(n / \varepsilon)$ | Approximation |

**Légende:** $n$ = nombre d'items, $C$ = capacité, $k$ = itérations, $g$ = générations, $p$ = population, $i$ = itérations SA, $\varepsilon$ = paramètre d'approximation

### 7.2 Classification des Algorithmes

**Algorithmes Exacts:**
- Garantissent la solution optimale
- Brute Force: $O(2^n)$ - inutilisable pour $n > 25$
- DP: $O(n \cdot C)$ - pseudo-polynomial, dépend de la capacité
- Branch & Bound: Élagage efficace, souvent meilleur que $O(2^n)$ en pratique

**Algorithmes d'Approximation:**
- FPTAS garantit $(1-\varepsilon) \times OPT$ en temps polynomial
- Greedy ratio: approximation $\frac{1}{2} \times OPT$ garantie

**Métaheuristiques:**
- Pas de garantie théorique sur la qualité
- Genetic Algorithm et Simulated Annealing explorent l'espace de recherche
- Bons résultats en pratique, surtout pour grandes instances

In [ ]:
# 7.3 Vérification Expérimentale de la Complexité
# On vérifie si le temps d'exécution suit la complexité théorique

if results_df is not None:
    from scipy import stats
    
    df = results_df.copy()
    
    # Tableau de complexité théorique
    complexity_info = {
        'Brute Force': {'type': 'Exponentiel', 'formula': 'O(2^n)', 'expected': 'exponentiel'},
        'Dynamic Programming': {'type': 'Pseudo-poly', 'formula': 'O(n×C)', 'expected': 'linéaire en n×C'},
        'DP Top-Down': {'type': 'Pseudo-poly', 'formula': 'O(n×C)', 'expected': 'linéaire en n×C'},
        'Branch and Bound': {'type': 'Exponentiel', 'formula': 'O(2^n) pire cas', 'expected': 'variable'},
        'Greedy Ratio': {'type': 'Polynomial', 'formula': 'O(n log n)', 'expected': 'quasi-linéaire'},
        'Greedy Value': {'type': 'Polynomial', 'formula': 'O(n log n)', 'expected': 'quasi-linéaire'},
        'Greedy Weight': {'type': 'Polynomial', 'formula': 'O(n log n)', 'expected': 'quasi-linéaire'},
        'Fractional Knapsack': {'type': 'Polynomial', 'formula': 'O(n log n)', 'expected': 'quasi-linéaire'},
        'Randomized': {'type': 'Polynomial', 'formula': 'O(k×n)', 'expected': 'linéaire'},
        'Genetic Algorithm': {'type': 'Polynomial', 'formula': 'O(g×p×n)', 'expected': 'linéaire'},
        'Genetic Adaptive': {'type': 'Polynomial', 'formula': 'O(g×p×n)', 'expected': 'linéaire'},
        'Simulated Annealing': {'type': 'Polynomial', 'formula': 'O(i×n)', 'expected': 'linéaire'},
        'SA Adaptive': {'type': 'Polynomial', 'formula': 'O(i×n)', 'expected': 'linéaire'},
        'FTPAS (ε=0.1)': {'type': 'Polynomial', 'formula': 'O(n²/ε)', 'expected': 'quadratique'},
        'FTPAS (ε=0.05)': {'type': 'Polynomial', 'formula': 'O(n²/ε)', 'expected': 'quadratique'},
        'FTPAS Adaptive': {'type': 'Polynomial', 'formula': 'O(n²/ε)', 'expected': 'quadratique'},
    }
    
    # Créer le tableau récapitulatif
    summary_data = []
    
    for algo in df['algorithm'].unique():
        algo_df = df[df['algorithm'] == algo]
        
        if len(algo_df) < 3:
            continue
        
        # Statistiques
        avg_time = algo_df['time_ms'].mean()
        min_n = algo_df['n'].min()
        max_n = algo_df['n'].max()
        
        # Régression log-log pour estimer l'exposant
        # time ∝ n^k => log(time) = k*log(n) + c
        valid_data = algo_df[algo_df['time_ms'] > 0]
        if len(valid_data) >= 3:
            log_n = np.log(valid_data['n'].values)
            log_t = np.log(valid_data['time_ms'].values)
            slope, intercept, r_value, p_value, std_err = stats.linregress(log_n, log_t)
            growth_rate = f"n^{slope:.2f}"
            r_squared = r_value**2
        else:
            growth_rate = "N/A"
            r_squared = 0
        
        info = complexity_info.get(algo, {'type': '?', 'formula': '?', 'expected': '?'})
        
        summary_data.append({
            'Algorithme': algo,
            'Type': info['type'],
            'Complexité': info['formula'],
            'Croissance mesurée': growth_rate,
            'R²': f"{r_squared:.3f}" if r_squared > 0 else "N/A",
            'Temps moyen (ms)': f"{avg_time:.2f}",
            'Plage n': f"{min_n}-{max_n}"
        })
    
    complexity_df = pd.DataFrame(summary_data)
    
    print("=" * 90)
    print("ANALYSE DE COMPLEXITÉ - COMPARAISON THÉORIQUE VS EXPÉRIMENTALE")
    print("=" * 90)
    print()
    print(complexity_df.to_string(index=False))
    print()
    print("Note: La 'Croissance mesurée' est estimée par régression log-log (time ∝ n^k)")
    print("      R² proche de 1 indique une bonne corrélation avec le modèle polynomial")
else:
    print("Exécutez d'abord les benchmarks pour voir l'analyse de complexité")

In [ ]:
# 7.4 Graphique de Croissance du Temps d'Exécution (échelle log-log)

if results_df is not None:
    df = results_df.copy()
    
    # Sélectionner quelques algorithmes représentatifs
    algos_to_plot = ['Dynamic Programming', 'Greedy Ratio', 'Genetic Adaptive', 'SA Adaptive', 'Branch and Bound']
    algos_available = [a for a in algos_to_plot if a in df['algorithm'].unique()]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # --- Graphique 1: Temps vs n (log-log) ---
    ax1 = axes[0]
    
    for algo in algos_available:
        algo_df = df[df['algorithm'] == algo].groupby('n')['time_ms'].mean().reset_index()
        algo_df = algo_df.sort_values('n')
        
        color = ALGO_COLORS.get(algo, '#333333')
        ax1.loglog(algo_df['n'], algo_df['time_ms'], 'o-', label=algo, 
                   color=color, linewidth=2, markersize=6)
    
    # Lignes de référence pour les complexités
    n_ref = np.array([10, 100, 1000, 10000])
    ax1.loglog(n_ref, n_ref * 0.001, '--', color='gray', alpha=0.5, label=r'$O(n)$')
    ax1.loglog(n_ref, n_ref**2 * 0.00001, '-.', color='gray', alpha=0.5, label=r'$O(n^2)$')
    ax1.loglog(n_ref, n_ref * np.log(n_ref) * 0.001, ':', color='gray', alpha=0.5, label=r'$O(n \log n)$')
    
    ax1.set_xlabel('Taille n (log)', fontsize=11)
    ax1.set_ylabel('Temps (ms, log)', fontsize=11)
    ax1.set_title('Croissance du temps d\'exécution (échelle log-log)')
    ax1.legend(fontsize=8, loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # --- Graphique 2: Distribution des temps par type d'algorithme ---
    ax2 = axes[1]
    
    # Grouper par type
    algo_types = {
        'Exact': ['Brute Force', 'Dynamic Programming', 'DP Top-Down', 'Branch and Bound'],
        'Greedy': ['Greedy Ratio', 'Greedy Value', 'Greedy Weight', 'Fractional Knapsack'],
        'Métaheuristique': ['Genetic Algorithm', 'Genetic Adaptive', 'Simulated Annealing', 'SA Adaptive', 'Randomized'],
        'Approximation': ['FTPAS (ε=0.1)', 'FTPAS (ε=0.05)', 'FTPAS Adaptive']
    }
    
    type_times = []
    type_names = []
    
    for type_name, algos in algo_types.items():
        times = df[df['algorithm'].isin(algos)]['time_ms'].values
        if len(times) > 0:
            type_times.append(times)
            type_names.append(type_name)
    
    bp = ax2.boxplot(type_times, labels=type_names, patch_artist=True)
    colors_box = ['#e41a1c', '#4daf4a', '#377eb8', '#984ea3']
    for patch, color in zip(bp['boxes'], colors_box):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    
    ax2.set_ylabel('Temps (ms)', fontsize=11)
    ax2.set_title('Distribution des temps par type d\'algorithme')
    ax2.set_yscale('log')
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("\nInterprétation:")
    print("- Pente ~1 en log-log → complexité O(n)")
    print("- Pente ~2 en log-log → complexité O(n²)")
    print("- Les algorithmes Greedy sont les plus rapides (quasi-constants)")
    print("- Les métaheuristiques ont un temps prévisible mais plus élevé")
else:
    print("Exécutez d'abord les benchmarks")

### 7.5 Analyse de la Complexité - Conclusions

**Observations clés:**

1. **Algorithmes polynomiaux vs exponentiels:**
   - Greedy ($O(n \log n)$) reste très rapide même pour $n = 10000$
   - Brute Force ($O(2^n)$) explose dès $n > 20$
   - DP ($O(n \cdot C)$) est pseudo-polynomial: dépend aussi de la capacité

2. **Impact du paramètre $\varepsilon$ sur FPTAS:**
   - $\varepsilon = 0.05$: meilleure qualité mais $O(n^2/0.05) = O(20n^2)$
   - $\varepsilon = 0.1$: plus rapide avec $O(n^2/0.1) = O(10n^2)$
   - Trade-off qualité/temps contrôlable

3. **Métaheuristiques:**
   - Temps quasi-linéaire en $n$ (paramètres fixés)
   - Qualité variable mais souvent proche de l'optimal
   - Simulated Annealing généralement plus rapide que Genetic Algorithm

4. **Recommandations pratiques:**
   - $n \leq 20$: Brute Force acceptable
   - $n \leq 1000$ et $C$ petit: DP optimal
   - $n$ grand: Greedy pour rapidité, métaheuristiques pour qualité